In [ ]:
!pip install -qU langchain-openai
!pip install nltk

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain.agents import AgentExecutor, Tool
from langchain.llms import OpenAI
import os
from google.colab import drive
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool

import nltk
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.schema import Document

os.environ["OPENAI_API_KEY"] = openai_api_key

drive.mount('/content/drive')

output_dir = "/content/drive/MyDrive/LLM Ass 3"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from langchain.vectorstores import FAISS

In [15]:
from langchain.agents import initialize_agent, create_openai_tools_agent
from langchain.agents import AgentType, create_react_agent

In [5]:
# import pdfplumber


file_path = "/content/drive/MyDrive/LLM Ass 3/prompting-12.pdf"
loader = PyPDFLoader(file_path)
text = loader.load()
# print(text)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(text)


In [6]:
#Create the embeddings and FAISS vector store
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(split_docs, embeddings)
retriever=vector_store.as_retriever()

In [7]:
### Build retriever tool ###
qa_tool = create_retriever_tool(
    retriever,
    "document_retriever",
    "Extracts information from the given documents.",
)

In [31]:


@tool
def extract_keywords(text: str) -> dict[str, list[str]]:
  """
    Extract keywords from a given text.

    This function tokenizes the input text, removes stopwords, and keeps only alphanumeric words,
    returning a list of keywords in lowercase.

    Parameters:
    - text (str): The input text from which to extract keywords.

    Returns:
    - dict: A dictionary with a single key "keywords", where the value is a list of keywords extracted from the input text.
      Example: {"keywords": ["example", "keyword", "list"]}
    """
  words = word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  keywords = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
  return {"keywords": keywords}



@tool
def frequency_counter(keywords: list[str]) -> dict[str, dict[str, int]]:
  """
    Count the frequency of each keyword in a list.

    This function takes a list of keywords and calculates the frequency of each keyword,
    returning a dictionary of keyword frequencies.

    Parameters:
    - keywords (List[str]): A list of keywords to analyze for frequency.

    Returns:
    - dict: A dictionary with a single key "frequency_dict", where the value is another dictionary
      mapping each keyword to its frequency count.
      Example: {"frequency_dict": {"example": 3, "keyword": 5, "list": 2}}
    """
  frequency = Counter(keywords)
  return {"frequency_dict": dict(frequency)}


@tool
def top_keywords(frequency_dict: dict[str, int]) -> dict[str, list[str]]:
  """
    Identify the top 10 most frequent keywords.

    This function takes a frequency dictionary of keywords (that is the frequency_dict output of the frequency_counter function) and sorts them in descending order by frequency.
    It returns the top 10 keywords.

    Parameters:
    - frequency_dict (Dict[str, int]): A dictionary where keys are keywords and values are their frequency counts.

    Returns:
    - dict: A dictionary with a single key "top_keywords", where the value is a list of the top 10 keywords by frequency.
      Example: {"top_keywords": ["most", "frequent", "keywords"]}
    """
  top_keywords = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=True)[:10]
  return [keyword for keyword, freq in top_keywords]

tools = [extract_keywords, frequency_counter, top_keywords]

In [9]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [32]:
# #Create the memory for chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


llm = ChatOpenAI(model="gpt-4o-mini")

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Follow these steps automatically: "
                   "1) Extract keywords from the document. "
                   "2) Calculate the frequency of each keyword. "
                   "3) Identify the top 10 most frequent keywords. "
                   "4) Use those 10 keywords to generate a poem."),
        ("human", "{input}"),
        ("assistant", "{agent_scratchpad}")
    ]
)


# prompt = PromptTemplate(template="""You are a helpful assistant answering questions based on documents.
# Available tools: {tool_names}
# {tools}

# Question: {input}
# ("assistant", "{agent_scratchpad}")""")
# Create the agent executor (react agent). Checkout the documentation for more type of agents.


# Start the conversation with a system message to set context
agent = create_openai_tools_agent(llm, tools, chat_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke(
        {
            "input": f"""
            create a poem based on the the ten most frequently occurring words in the {text}"""
        }
    )



> Entering new AgentExecutor chain...

Invoking: `extract_keywords` with `{'text': "Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2024. All\nrights reserved. Draft of August 20, 2024.\nCHAPTER\n12\nModel Alignment, Prompting,\nand In-Context Learning\n“Hal, ” said Bowman, now speaking with an icy calm. “I am not incapacitated. Unless you obey my instructions, I shall be forced to disconnect you. ”\nArthur C. Clarke\nIn this chapter we show how to get LLMs to do tasks for us simply by talking to them. To get an LLM to translate a sentence, outline a talk, or draft a work email,\nwe’ll simply describe what we want in natural language. We call these instructions we give to language models prompts.prompts\nPrompting relies on contextual generation. Given the prompt as context, the lan-\nguage model generates the next token based on its token probability, conditioned on\nthe prompt: P(wi|w<i). A prompt can be a question (like “What is a transformer net-\nw

In [34]:

print(result['output'])


Here is a poem crafted using the top 10 most frequent keywords extracted from the document:

---

In the realm of **language**, vast and wide,  
A **model** learns, with knowledge as its guide.  
Through **instructions** clear, it finds its way,  
**Prompting** thoughts that dance and play.  

With every **task**, a new journey begins,  
In the heart of **LLMs**, where the magic spins.  
**Learning** unfolds like a blossoming flower,  
In the world of AI, they harness their power.  

As we delve into the art of **tuning**,  
With each **like**, we find joy in the blooming.  
Together we explore this intricate dance,  
In the landscape of knowledge, we take our chance.  

**Hal** once whispered, with caution and fear,  
But now we embrace what we hold dear.  
In this digital age, let us take flight,  
With models that learn, we reach for the light.  

--- 

I hope you enjoyed the poem!
